In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv("../training_big.dat", dtype=np.float32)

In [4]:
arr = np.array(df)

In [93]:
df = pd.read_csv("../test.dat")
arr = np.array(df)
arr.shape

(2, 677)

In [12]:
X = arr[:, :676]
Y = arr[:, -1]
order = np.arange(len(X))
np.random.shuffle(order)
X = X[order]
Y = Y[order]
N = len(X)
train_test_split = 0.8
N_X = int(train_test_split * N)
x_train = X[:N_X]
y_train = Y[:N_X]
x_test = X[N_X:]
y_test = Y[N_X:]

KeyboardInterrupt: 

In [18]:
inputs = tf.keras.Input(shape=(676,))
x = tf.keras.layers.Dense(100, activation = tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs = inputs, outputs = outputs)

In [28]:
model.compile(loss=tf.keras.losses.MSE,metrics=["accuracy"] )

In [66]:
def write_weights(model, file):
    with open(file, 'w') as f:
        weights = model.get_weights()
        for i in range(len(weights)):
            w = weights[i]
            if i % 2 == 0:
                layer_desc = "WEIGHTS"
                layer_type = "[f32;{}]".format(w.shape[0] * w.shape[1])
                s = "pub const LAYER_{}_WEIGHTS_SHAPE: (usize, usize) = ({},{});\n".format(i//2, w.shape[0],w.shape[1])
            else:
                layer_desc = "BIAS"
                layer_type = "[f32;{}]".format(len(w))
                s = ""

            s += "#[rustfmt::skip]\npub const LAYER_{}_{} : {} = ".format(i//2,layer_desc,layer_type)
            s += "[";
            for j in range(len(w)):
                if i % 2 == 0:
                    for k in range(len(w[j])):
                        s += str(w[j,k])
                        if k != len(w[j]) - 1:
                            s += ","
                else:
                    s += str(w[j])
                s += ","
            s += "];\n"
            f.write(s)

In [99]:
model = tf.keras.models.load_model("second_model.h5")

In [101]:
write_weights(model, save+".weights")

In [80]:
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [100]:
model.predict(arr[:, :676])

array([[0.48872244],
       [0.58495736]], dtype=float32)

In [ ]:
save = "second_model"
import gc
import time
for i in range(100):
    gc.collect()
    time.sleep(5)
    model.fit(X[:N_X], Y[:N_X], validation_data=(X[N_X:],Y[N_X:]), epochs = 1)
    model.save(save+".h5")
#write_weights(model, save+".weights")

24845/62500 [==========>...................] - ETA: 42s - loss: 0.0626 - accuracy: 0.7312